
# Modelos de Clasificación para Predecir Sobrevivientes del Titanic

![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/class3/figures/ml-claro-2/titanic.jpg)


En general los conceptos del universo de problemas de regresión se trasladan al caso de clasificación con pequeñas diferencias.


Nuestro *target* es ahora una variable discreta o categórica de forma que el objetivo es clasificar etiquetas desconocidas en distintas clases.

Numerosos problemas pueden modelarse de este modo (rotulado de correo basura, churn, probabilidad de default, etc.).


Queremos que nuestro clasificador estimado, $\hat{f}$, minimice ahora la siguiente función:

$$\frac{1}{p}\sum_{i=1}^{p}I(y_{i} \not = \hat{y}_{i})$$
     
Donde $I(\cdot)$ es una función que vale 1 si $y_{i} \not = \hat{y}_{i}$ y 0 en caso contrario.

Básicamente deseamos reducir la proporción de errores que cometemos.

Esencialmente estamos intentando buscar una frontera de decisión (en el sentido que minimice el error de arriba)

Algunos clasificadores conocidos: Bayes ingenuo, k-vecinos más cercanos, regresión logística, etc.

## Extracción de datos para clasificación

### Ejercicios
1. Escriba una función que obtenga, a partir de [http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.csv](http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.csv) y utilizando Pandas, el conjunto de datos de pasajeros involucrados en el hundimiento del Titanic y lo guarde en un un archivo `titanic_local.csv`

2. Agregue lógica de cache que reuse la copia local del conjunto de datos, en caso que esta exista, y evite de ese modo consultar repetidas veces el enlace de arriba.

3. Parta este conjunto de datos de forma aleatoria en dos partes de manera tal que un subconjunto tenga el 70\% de los registros.

4. ¿Cómo puede garantizar que lo hecho en el inciso anterior sea replicable?  Fije la semilla del generador de números aleatorios en `1 2 3 4`.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

In [ ]:
def extract_titanic_data(url, refresh_cache=False):
    cache_fn = Path('titanic.csv')
    if not cache_fn.exists() or refresh_cache:
        print("Getting data")
        df = pd.read_csv(url)
        df.to_csv(cache_fn, index=False)
    else:
        print("Using cache")
        df = pd.read_csv(cache_fn)
    return df

In [ ]:
url = 'http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.csv'
df_raw = extract_titanic_data(url)
df_raw

In [ ]:
# Split data en train y holdout
np.random.seed(1234)
msk = np.random.rand(len(df_raw)) >= 0.3
df_train = df_raw[msk]
df_test = df_raw[~msk]

In [ ]:
df_train.head()

In [ ]:
len(df_train)

In [ ]:
df_test.head()

In [ ]:
df_train.columns

## Primer preproceso / EDA


![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/class3/figures/ml-claro-2/table_variables.png)

### Ejercicios

1. Escriba una función que permita convertir (*castear*) múltiples columnas de un tipo de dato a otro y utilícela para asegurar que sus datos sean consistentes con la tabla arriba.

2. ¿Qué variables puede usted ignorar? ¿Por qué? Escriba una función que reciba un dataframe y una lista de columnas a eliminar y devuelva un el data frame sin estas columnas. Loguee cuáles columnas fueron eliminadas.

3. Una los conjuntos de entrenamiento y validación en único dataframe con una columna de booleanos indicando pertenencia al conjunto de entrenamiento. ¿Por qué tiene sentido trabajar con un set de datos de esta forma?

4. ¿Tiene clases desbalanceadas? Haga un gráfico de barras para responder esta pregunta.

In [ ]:
df_train.info()

In [ ]:
# Tiene sentido dropear boat (y tambien body)
survived_with_boat = len(df_train[(~df_train['boat'].isnull()) & (df_train['survived'] == 1)])
survived = len(df_train[df_train['survived'] == 1])
(survived_with_boat / survived) * 100

In [ ]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s-%(name)s-%(levelname)s: %(message)s',
    handlers=[logging.FileHandler('titanic.log'), logging.StreamHandler()],
)
logger = logging.getLogger(__name__)

In [ ]:
def _drop_unusable_cols(df, cols=[]):
    logger.info(
        f"Dropping the following {len(cols)} unusable columns:\n"
        f"{cols}"
    )
    #df.drop(cols, axis=1, inplace=True)
    df = df.drop(cols, axis=1)
    logger.info(
        f"Remaining {len(df.columns)} columns:\n {sorted(df.columns.tolist())}"
    )
    return df

In [ ]:
df_train = _drop_unusable_cols(df_train, cols=['boat', 'body'])

In [ ]:
df_train.columns, df_test.columns

In [ ]:
df_test = _drop_unusable_cols(df_test, cols=['boat', 'body'])

In [ ]:
# Join train test
df_train['train'] = True
df_test['train'] = False
df = pd.concat([df_train, df_test])

In [ ]:
df

In [ ]:
df.columns

In [ ]:
pd.value_counts(df['sex'])

In [ ]:
%matplotlib inline
pd.value_counts(df['survived'], normalize=True).plot.bar()

## EDA

Vamos a utilizar `Seaborn` para hacer algunas visualizaciones estadísticas.

![](https://www.fromthegenesis.com/wp-content/uploads/2018/11/seaborn.jpg)


Seaborn es una librería de visualización de datos de Python basada en `matplotlib`. 

Proporciona una interfaz de alto nivel para dibujar gráficos estadísticos atractivos e informativos.

Tiene como objetivo hacer que la visualización sea una parte central de la exploración y comprensión de los datos. 


Veamos primero correlaciones de los atributos numéricos.


In [ ]:
import seaborn as sns

In [ ]:
g = sns.heatmap(df[['survived', 'age', 'parch', 'fare', 'sibsp']].corr(),
                annot=True, fmt = ".2f", cmap = "coolwarm")

Solamente el precio del boleto parece estar correlacionado con la probabilidad de supervivencia... ¿debemos descartar las otras variables?

In [ ]:
g = sns.FacetGrid(df, col='survived')
g = g.map(sns.distplot, 'age')

In [ ]:
g = sns.kdeplot(df['age'][(df['survived'] == 0) & 
                             (df['age'].notnull())], color='Red', shade = True)
g = sns.kdeplot(df['age'][(df['survived'] == 1) & 
                             (df['age'].notnull())], color='Blue', shade = True)
g.set_xlabel('age')
g.set_ylabel('Frequency')
g = g.legend(['Not Survived', 'Survived'])

### Ejercicios: Visualizaciones como herramienta de EDA

1. Use la función distplot para graficar la distribución del precio de boletos.
  1. ¿Es la distribución resultante asimétrica/sesgada? En caso afirmativo aplique alguna transformación sobre la variable y grafique la nueva distribución.
2. ¿Son los hombres o las mujeres más propensos a sobrevivir? Grafique la
probabilidad de supervivencia para cada caso y compute el valor exacto de las
mismas.
3. ¿Hay alguna clase del barco que garantice mayor probabilidad de supervivencia?
¿Es este resultado robusto a controlar por sexo? Hint: use la función catplot

In [ ]:
df[df['fare'].isnull()]

In [ ]:
df['fare'].mean()

In [ ]:
df[df['fare'].fillna(df['fare'].mean()).isnull()]

In [ ]:
# Distribución de precio de boletos 
g = sns.distplot(df['fare'].fillna(df['fare'].mean()), color='m')

In [ ]:
df[df['fare'] == 0]

In [ ]:
df['fare'] = df['fare'].map(lambda i: np.log(i) if i > 0 else 0)
g = sns.distplot(df['fare'].fillna(df['fare'].mean()), color='m')

In [ ]:
g = sns.barplot(x='sex', y='survived', data=df)
g = g.set_ylabel("Survival Probability")

In [ ]:
df[['sex', 'survived']].groupby('sex').mean()

In [ ]:
g = sns.catplot(x='pclass', y='survived', hue='sex', data=df,
                   height=6, kind='bar')
g = g.set_ylabels("survival probability")

## Valores faltantes, repetidos, constantes y extremos

1. ¿Cuál es la frecuencia de valores nulos o faltantes? ¿Vale la pena descartar por esto motivo alguna variable? ¿Imputamos valores? ¿Qué valor usar?

2. ¿Tienen contenido informativo aquellas variable con nula o cuasi-nula varianza?

3. ¿Tiene sentido reemplazar valores extremos?


## Ejercicios : Trabajando con valores nulos y constantes
1. Escriba una función que i) para cada atributo compute la proporción de valores nulos y ii) en caso que esta supere un determinado umbral elimine dicha columna.
  1. Potencialmente podría borrar información importantes (como el target!). ¿Cómo puede modificar la función anterior para proteger a esta columnas?

2. En espíritu similar al punto anterior escriba una función que elimine, si las hay, columnas con nula o cuasi nula varianza.

3. Escriba una función o lógica que permita rellenar valores de atributos numéricos por su mediana. ¿Cómo puede extender esto a atributos categóricos? Hint: le puede primero servir escribir una función auxiliar que identifique las columnas por tipo (categórica o numérica)

4. Sugiera/piense mejoras sobre el procesamiento hecho en los puntos anteriores.

In [ ]:
df.isnull().mean()

In [ ]:
df.info()

In [ ]:
(df.isnull().mean() < 0.5).index.tolist() 

In [ ]:
def _drop_nulls(df, max_null_prop=0.5):
    logger.info(
        f"Dropping columns with null ratio greater than {max_null_prop * 100}%..."
    )
    null_means = df.isnull().mean()
    null_mask = null_means < max_null_prop
    null_mask[[c for c in null_mask.index.tolist() if c in PROTECTED_COLS]] = True
    drop_cols = null_mask[~null_mask].index.tolist()
    logger.info(
        f"null proportions:\n"
        f"{null_means.loc[drop_cols].sort_values(ascending=False)}"
    )

    logger.info(f"Dropping the following {len(drop_cols)} columns:\n {drop_cols}")

    df = df.drop(drop_cols, axis=1)
    return df

In [ ]:
PROTECTED_COLS = ['survived', 'train']
df = _drop_nulls(df)

In [ ]:
df.info()

In [ ]:
df.std()

In [ ]:
def _drop_std(df, min_std_dev=1.5e-2):
    std_values = df.std()
    low_variance_cols = std_values < min_std_dev
    low_variance_cols = low_variance_cols.index[low_variance_cols].tolist()
    low_variance_cols = [c for c in low_variance_cols if c not in PROTECTED_COLS]
    logger.info(
        f'Dropping the following {len(low_variance_cols)} columns '
        f'due to low variance:\n {low_variance_cols}'
    )
    df.drop(low_variance_cols, axis=1, inplace=True)
    return df

In [ ]:
df = _drop_std(df)

In [ ]:
df.info()

In [ ]:
df.embarked.value_counts(dropna=False)

In [ ]:
def _get_typed_cols(df, col_type='cat'):
    assert col_type in ('cat', 'num')
    include = 'object' if col_type == 'cat' else [np.number]
    typed_cols = [
        c for c in df.select_dtypes(include=include).columns if c not in PROTECTED_COLS
    ]
    return typed_cols

In [ ]:
num_cols = _get_typed_cols(df, col_type='num')
cat_cols = _get_typed_cols(df, col_type='cat')
num_cols, cat_cols

In [ ]:
df['sex'].value_counts().index.to_list()[0]

In [ ]:
def _fill_nulls(df):
    for t in ['num', 'cat']:
        cols = _get_typed_cols(df, col_type=t)
        for c in cols:
            if t == 'num':
                df[c] = df[c].fillna(df[c].median())
            else:
                val_count = df[c].value_counts(dropna=True)
                common_val = val_count.index.tolist()[0]
                df[c] = df[c].fillna(common_val)
    return df

In [ ]:
df = _fill_nulls(df)

In [ ]:
df.info()

In [ ]:
df.embarked.value_counts(dropna=False)


## "*Applied Machine Learning is basically Feature Engineering*" - Andrew Ng


## Ingeniería de Atributos (Feature Engineering)
### Ejercicios

1. Cree un nuevo atributo family_size con el tamaño de la familia de cada pasajero (incluyendo a el mismo).
2. Dado este nuevo atributo genere 3 atributos adicionales que remitan a familias de un único miembro, de 2 a 4 miembros y mayor o igual a 5.
3. Haga uno (o varios) gráficos comparando la probabilidad de supervivencia de cada una de estas familias.
4. Bonus: Extraiga un prefijo a partir de la columna de boletos siempre y cuando el valor de esa columna no sea numerico. Reemplace esta columna por dicho prefijo.

In [ ]:
df.name

In [ ]:
df.name.str.split(',').str[-1].str.split('.').str[0].str.strip().value_counts()

In [ ]:
# Creamos un nuevo atributo "titulo"
df['title'] = df.name.str.split(',').str[-1].str.split('.').str[0].str.strip()
df['title'] = df['title'].replace(
    df.title.value_counts(dropna=False).index.tolist()[4:], 'other'
)
df['title'] = df['title'].replace(['Miss'], 'Mrs')
df = df.drop(['name'], axis=1)

In [ ]:
df

In [ ]:
df.title.value_counts()

In [ ]:
g = sns.countplot(df['title'])

In [ ]:
g = sns.catplot(x='title',y='survived',data=df,kind="bar")

In [ ]:
df.info()

In [ ]:
df['family_size'] = df['parch'] + df['sibsp'] + 1
df['family_single'] = df['family_size'] == 1
df['family_small'] = (df['family_size'] > 1) & (df['family_size'] <= 4)
df['family_large'] = df['family_size'] > 4

In [ ]:
df.sample(20)

In [ ]:
for fsize in ['single', 'small', 'large']:
    g = sns.catplot(x=f'family_{fsize}',y='survived',data=df,kind="bar")
    g = g.set_ylabels("Survival Probability")

In [ ]:
df.ticket.sample(30)

In [ ]:
# Ticket prefix
def extract_ticket_prefix(i):
    if not i.isdigit() :
        rv = i.replace('.',"").replace('/',"").strip().split(' ')[0]
    else:
        rv = 'X'
    return rv

In [ ]:
df['ticket'] = df['ticket'].apply(extract_ticket_prefix)

In [ ]:
df.ticket.value_counts()

In [ ]:
df.info()

## Regresión Logística

Supongamos que queremos modelar la probabilidad que un cliente cancele o no su línea de celular

Podriamos pensar en un modelo de regresión lineal como los ya vistos:
          $$Y_{t} = \beta_{0} + \beta_{1}X_{t-k} + \varepsilon_{t}$$

En base a esto definir un cliente "cancelador" si $\hat{Y} > 0.5$

Pero... el target no caerá necesariamente en el intervalo $[0,1]$.

Tiene sentido en cambio formular el problema como
        $$\Pr(Y_{t} = 1) = F(\beta_{0} + \beta_{1}X_{t-k} + \varepsilon_{t})$$

Donde $F(\cdot)$ es la función logística dada por $F(\cdot) = \frac{1}{1 + e^{-x}}$
    
```python
  >>> x = np.linspace(-5, 5, 100)
  >>> y = 1 / (1 + np.exp(-x))
  >>> plt.plot(x, y)
```
![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/class3/figures/ml-claro-1/logistic.png)


### Métricas de Peformance

En general estos modelos devuelven estimaciones de probabilidades condicionales.

¿Cómo sabemos si fueron buenas?

Necesitamos alguna métrica de evaluación. Una matriz de confusión nos permite definir algunas:

![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/class3/figures/ml-claro-1/confusion_matrix.png)

Hay dos tipos de errores: falsos positivos y falsos negativos

* Podemos derivar métricas en base a estos errores

*  La más “natural” se conoce como exactitud o accuracy definida por el ratio de clasificaciones correctas sobre el total realizado:

$$\frac{TP + TN}{TP + TN + FP + FN}$$

¿Es una buena métrica? Puede ser engañosas...


![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/class3/figures/ml-claro-1/unbalance_class.png)


Imaginemos el caso último panel con una proporción de 1 caso positivo cada 100

Un clasificador trivial que prediga siempre la clase negativa tiene una exactitud del 99 %.

¿Podemos construir una métrica para evitar esta situación? Hay que tratar de
evitar mezclar los verdaderos positivos y negativos...

### Precisión, cobertura y F1 Score

Para sortear los problemas anteriores es frecuente utilizar las siguiente dos métricas

**Precisión**: cantidad de casos correctamente rotulados como positivos sobre el total de predicciones positivas, $TP / (TP + FP)$.

**Cobertura o Recall**: proporción de instancias positivas que el algoritmo logra identificar sobre el total de casos positivos ($TP / (TP + FN)$)

Depende del contexto puede ser deseable maximizar una o la otra

Si es una enfermerdad rara, por caso, nos interesa más la cobertura que la precisión. 

**¿Para un filtro de spam?**


En determinadas situaciones ambos métricas son importantes y tiene sentido combinarlas:

$$F_{1} = 2\frac{p\cdot r}{p + r}$$

Pesa por igual a ambas métricas (media harmónica) y el valor suele estar cerca del mínimo de las métrica.

Esta acotado al intervalo $[0, 1]$ y vale 1 únicamente para un clasificador perfecto.

Valores superiores a 0.7 son propios de un "buen" clasificador.

Desventaja: no hace un juicio sobre como el modelo clasifica las instancias negativas

![](https://miro.medium.com/max/1336/1*uzJKEMrjHEv9DBAGNke3EQ.png)

### La curva ROC

![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/class3/figures/ml-claro-1/roc.png)

Características:

Para definir pertenencia a una clase hay un umbral, $p$, tal que si $p=0$ (1) siempre (nunca) predigo que las observaciones son positivas entonces ambos TPR (TP / P) y FPR (FP/ N) valen 1 (0).

Curva 45 grados define a un clasificador aleatorio y la coordenada $(0,1)$ a un clasificador perfecto.

Intuitivamente si elegimos aleatoriamente una observacion positiva y una negativa el area bajo la curva es la probabilidad de que el clasificador rankee más alto al positivo.

**Conviene utilizar una única métrica para guiar las decisiones**


## Fitteo del modelo

Ejercicios: Fit Regresión Logística
1. Alguno algoritmos, en Python, soportan exclusivamente atributos numéricos. Escriba en consecuencia una función para convertir todos los atributos categóricos. 
2. Fitee una regresión logística sobre la data de entrenamiento y genere predicciones (y probabilidades) para la data de test/validación.
3. Evalue estas predicciones usando métricas de accuracy y ROC para el conjunto de entrenamiento y ROC para el conjunto de validación.
4. Bonus: investigue que es la metrica F1 y utilice scikit-learn o escriba una función que compute dicha métrica y
con ello evalué nuevamente los resultados para ambos conjuntos

In [ ]:
df['home.dest'].value_counts().index.values[0: 10]

In [ ]:
def _encode_categorical(df, top=20):
    logger.info("Filtering categorical columns top values...")
    cat_cols = _get_typed_cols(df, col_type='cat')
    logger.info(f"Categorical columns:\n {cat_cols}")

    for c in cat_cols:
        top_categories = df[c].value_counts().index.values[0:top]
        logger.info(f"Top categories for {c}:\n {top_categories}")
        df[c] = df[c].where(df[c].isin(top_categories), other='OTHER')

    logger.info("Getting dummies from top categories...")
    df = pd.get_dummies(df, columns=cat_cols, drop_first=False)
    logger.info(
        f"{len(df.columns)} columns after dummies:\n " f"{sorted(df.columns.tolist())}"
    )
    return df

In [ ]:
df = _encode_categorical(df)

In [ ]:
df.columns

In [ ]:
df_train = df[df['train']]
df_test = df[~df['train']]
y_train = df_train['survived']
y_test = df_test['survived']
X_train = df_train.drop(['survived', 'train'], axis=1)
X_test = df_test.drop(['survived', 'train'], axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred_proba = lr.predict_proba(X_test)[:, 1]

In [ ]:
y_pred

In [ ]:
y_pred_proba

In [ ]:
lr.score?

In [ ]:
lr.score(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
import sklearn.metrics as metrics

In [ ]:
roc_score = metrics.roc_auc_score(y_test, y_pred_proba)
roc_score

In [ ]:
cr = metrics.classification_report(y_test, y_pred)
print(cr)

## Árboles de decisión y aleatorios

Ejercicios

1. Fittee un árbol de decisión en vez de una regresión logística y compare la
performance de los modelos.
  * Bonus: Grafique el árbol de decision resultante
2. Repita el punto anterior pero ahora con un bosque aleatorio.
  * Bonus: Pruebe aumentar la profundidad del árbol. ¿Mejora su métrica de
performance?
3. Compute y grafique la importancia de atributos.

In [ ]:
from sklearn import tree

In [ ]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
y_pred_proba = dt.predict_proba(X_test)[:, 1]

In [ ]:
tree.plot_tree(dt)

In [ ]:
dt.score(X_train, y_train)

In [ ]:
dt.score(X_test, y_test)

In [ ]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=7, n_jobs=-1, verbose=2,)
rf = rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:, 1]
print(rf.score(X_train, y_train))
print(rf.score(X_test, y_test))

In [ ]:
roc_score = metrics.roc_auc_score(y_test, y_pred_proba)
roc_score

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
feat_import = list(
zip(np.asanyarray(X_train.columns)[indices], importances[indices])
)
feat_import = pd.DataFrame(feat_import, columns=['feature', 'importance'])

In [ ]:
feat_import

In [ ]:
ax = feat_import[:20].plot(kind='bar')
ax.set_xticklabels(feat_import[:20]['feature'].tolist())

## Ejercicios Finales

1. Aplicando lo aprendido en este notebook desarrolle uno nuevo con lo que considere más relevante y utilizando el RandomForestClassifier y submitee sus predicciones a la competencia de datos del Titanic https://www.kaggle.com/c/titanic/overview

2. Revise en detalle alguno de los dos noteobooks e incopore mejoras a su notebook para mejorar la predicción:

  https://www.kaggle.com/masumrumi/a-statistical-analysis-ml-workflow-of-titanic

  https://www.kaggle.com/rp1611/step-by-step-tutorial-for-beginners

3. Pruebe los resultados utilziando algún estimador de Boosting como LightGBM. Qué diferencias observa?
